In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import SelectFromModel, VarianceThreshold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from time import time
import re
import seaborn as sns

# Configure pandas and numpy settings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
np.set_printoptions(precision=15)

# Set random seed for reproducibility
RANDOM_SEED = 280903571
np.random.seed(RANDOM_SEED)

In [2]:
class ChemistryAwareAnalysis:
    """
    A class for analyzing chemical reaction data using machine learning.
    Implements proper stratified sampling and XGBoost regression.
    """
    
    def __init__(self, embeddings_path, data_path):
        """
        Initialize the analysis with embeddings and data paths.
        Also initializes scalers for selectivity and ddG values.
        
        Args:
            embeddings_path (str): Path to the JSON file containing molecular embeddings
            data_path (str): Path to the CSV file containing reaction data
        """
        print(f"Initializing ChemistryAwareAnalysis...")
        self.embeddings = self.load_embeddings(embeddings_path)
        print(f"Loaded embeddings for {len(self.embeddings)} molecules")
        
        # Load reaction data
        self.Y_df = pd.read_csv(data_path, dtype={
            'catalyst_id': str,
            'imine_id': str,
            'thiol_id': str,
            'product_id': str
        })
        print(f"Loaded data with {len(self.Y_df)} entries")
        
        # Initialize and fit scalers on full dataset
        self.ee_scaler = MinMaxScaler()
        self.ddg_scaler = MinMaxScaler()
        
        self.ee_scaler.fit(self.Y_df['selectivity_ee_percent'].values.reshape(-1, 1))
        self.ddg_scaler.fit(self.Y_df['selectivity_ddGact_kcal'].values.reshape(-1, 1))
        
        # Print data ranges
        print("\nData Ranges (Original Scale):")
        print(f"ddG range: [{self.Y_df['selectivity_ddGact_kcal'].min():.2f}, "
              f"{self.Y_df['selectivity_ddGact_kcal'].max():.2f}] kcal/mol")
        print(f"ee% range: [{self.Y_df['selectivity_ee_percent'].min():.2f}, "
              f"{self.Y_df['selectivity_ee_percent'].max():.2f}]%")
    
    @staticmethod
    def load_embeddings(file_path):
        """
        Load and process embeddings from JSON file.
        
        Args:
            file_path (str): Path to JSON file containing embeddings
            
        Returns:
            dict: Processed embeddings with cleaned keys
        """
        print(f"Loading embeddings from {file_path}")
        with open(file_path, 'r') as f:
            raw_embeddings = json.load(f)
        
        # Clean embedding keys by removing family prefix
        embeddings = {}
        family_pattern = re.compile(r'^family\d+_')
        for key, value in raw_embeddings.items():
            stripped_key = family_pattern.sub('', key)
            embeddings[stripped_key] = np.array(value)
        
        return embeddings

    def prepare_data(self):
        """
        Prepare data including scaling of selectivity and ddG values.
        
        Returns:
            tuple: (X_data, Y_data, ee_values, reaction_handles)
        """
        print("\nPreparing data...")
        X_data = []
        Y_data = []
        ee_values = []
        reaction_handles = []
        
        missing_molecules = set()
        for _, row in self.Y_df.iterrows():
            molecule_ids = [
                row['catalyst_id'],
                row['imine_id'],
                row['thiol_id'],
                row['product_id']
            ]
            
            if all(id in self.embeddings for id in molecule_ids):
                # Combine embeddings for all molecules in reaction
                combined_embedding = np.concatenate([
                    self.embeddings[id] for id in molecule_ids
                ])
                X_data.append(combined_embedding)
                
                # Scale ddG and ee values to [0,1]
                scaled_ddg = self.ddg_scaler.transform([[row['selectivity_ddGact_kcal']]])[0][0]
                scaled_ee = self.ee_scaler.transform([[row['selectivity_ee_percent']]])[0][0]
                
                Y_data.append(scaled_ddg)
                ee_values.append(scaled_ee)
                reaction_handles.append(row['reaction_handle'])
            else:
                missing_ids = [id for id in molecule_ids if id not in self.embeddings]
                missing_molecules.update(missing_ids)
        
        print(f"Prepared {len(X_data)} samples")
        if missing_molecules:
            print(f"Missing embeddings for {len(missing_molecules)} molecules: {missing_molecules}")
        print(f"Embedding dimension: {X_data[0].shape}")
        
        return (np.array(X_data), np.array(Y_data), 
                np.array(ee_values), np.array(reaction_handles))

    def split_data_stratified(self, X, Y, ee_values, reaction_handles, 
                            test_fraction=0.25, n_bins=5):
        """
        Proper stratified split ensuring representation across energy range.
        
        Args:
            X (np.ndarray): Feature matrix
            Y (np.ndarray): Target values (scaled energies)
            ee_values (np.ndarray): Scaled selectivity values
            reaction_handles (np.ndarray): Reaction identifiers
            test_fraction (float): Fraction of data for test set
            n_bins (int): Number of energy bins for stratification
            
        Returns:
            dict: Train/test split data
        """
        print("\nPerforming improved stratified split...")
        
        # Create energy bins
        Y_bins = pd.qcut(Y, n_bins, labels=False)
        
        train_indices = []
        test_indices = []
        
        print("\nBin statistics:")
        for bin_idx in range(n_bins):
            bin_mask = Y_bins == bin_idx
            bin_indices = np.where(bin_mask)[0]
            
            # Get original scale energy values for this bin
            bin_energies = self.ddg_scaler.inverse_transform(
                Y[bin_indices].reshape(-1, 1)).flatten()
            
            print(f"\nBin {bin_idx}:")
            print(f"Energy range: {bin_energies.min():.2f} to {bin_energies.max():.2f} kcal/mol")
            print(f"Number of samples: {len(bin_indices)}")
            
            # Calculate number of test samples for this bin
            n_test = int(len(bin_indices) * test_fraction)
            
            # Sort indices by ee values within bin
            sorted_by_ee = sorted(bin_indices, key=lambda x: ee_values[x])
            
            # Take balanced samples for test set
            test_indices.extend(sorted_by_ee[-n_test//2:])  # High ee
            test_indices.extend(sorted_by_ee[:n_test//2])   # Low ee
            
            # Remaining samples go to train
            train_indices.extend(sorted_by_ee[n_test//2:-n_test//2])
            
            print(f"Train samples: {len(sorted_by_ee[n_test//2:-n_test//2])}")
            print(f"Test samples: {n_test}")
        
        # Convert to arrays
        train_indices = np.array(train_indices)
        test_indices = np.array(test_indices)
        
        # Print final split statistics
        print("\nFinal split statistics:")
        print(f"Training set size: {len(train_indices)}")
        print(f"Test set size: {len(test_indices)}")
        
        # Print energy ranges
        train_energies = self.ddg_scaler.inverse_transform(
            Y[train_indices].reshape(-1, 1)).flatten()
        test_energies = self.ddg_scaler.inverse_transform(
            Y[test_indices].reshape(-1, 1)).flatten()
        
        print(f"Training energy range: {train_energies.min():.2f} to {train_energies.max():.2f} kcal/mol")
        print(f"Test energy range: {test_energies.min():.2f} to {test_energies.max():.2f} kcal/mol")
        
        return {
            'X_train': X[train_indices],
            'X_test': X[test_indices],
            'Y_train': Y[train_indices],
            'Y_test': Y[test_indices],
            'train_handles': reaction_handles[train_indices],
            'test_handles': reaction_handles[test_indices],
            'train_ee': ee_values[train_indices],
            'test_ee': ee_values[test_indices]
        }
    
    def train_and_evaluate(self, split_data, split_name=""):
        """
        Train XGBoost model and evaluate performance.
        Handles scaling/unscaling of values and comprehensive parameter tuning.
        
        Args:
            split_data (dict): Train/test split data
            split_name (str): Name identifier for the split
            
        Returns:
            dict: Results including model performance metrics and predictions
        """
        # Initialize pipeline with XGBoost
        pipe_random = Pipeline(steps=[
            ('preprocess', VarianceThreshold(1e-3)),
            ('feature_selection', SelectFromModel(
                RandomForestRegressor(
                    n_estimators=1000, 
                    n_jobs=64, 
                    random_state=RANDOM_SEED
                ),
                max_features=30
            )),
            ('model', XGBRegressor(
                objective='reg:squarederror',
                random_state=RANDOM_SEED,
                n_jobs=64,
                tree_method='hist'  # Faster histogram-based algorithm
            ))
        ])

        # Comprehensive XGBoost parameter grid
        param_dict = {
            # Basic parameters
            'model__learning_rate': np.logspace(-4, -1, 20),  # 0.0001 to 0.1
            'model__n_estimators': [100, 200, 300, 400, 500, 750, 1000],
            'model__max_depth': [3, 4, 5, 6, 7, 8],
            
            # Sampling parameters
            'model__subsample': np.linspace(0.6, 1.0, 5),         # Row sampling
            'model__colsample_bytree': np.linspace(0.6, 1.0, 5),  # Column sampling
            
            # Regularization parameters
            'model__reg_alpha': np.logspace(-4, 1, 10),     # L1 regularization
            'model__reg_lambda': np.logspace(-4, 1, 10),    # L2 regularization
            'model__min_child_weight': [1, 2, 3, 4, 5],     # Minimum sum of weights
            'model__gamma': [0, 0.1, 0.2, 0.3, 0.4, 0.5],   # Minimum loss reduction
        }

        # Random search with cross-validation
        search = RandomizedSearchCV(
            estimator=pipe_random,
            param_distributions=param_dict,
            n_iter=100,  # Number of parameter settings sampled
            cv=5,        # 5-fold cross-validation
            n_jobs=64,   # Parallel processing
            verbose=3,
            scoring=['neg_mean_absolute_error', 'r2'],
            refit='neg_mean_absolute_error',
            random_state=RANDOM_SEED
        )

        print(f"\nTraining XGBoost model with {split_name} split...")
        print("Model configuration:")
        print(f"- Cross-validation: 5-fold")
        print(f"- Parameter samples: 50")
        print(f"- Optimization metric: MAE")
        
        # Time the training
        t0 = time()
        search.fit(split_data['X_train'], split_data['Y_train'])
        training_time = time() - t0
        print(f"\nTraining completed in {training_time:.2f} seconds")

        # Get best model and parameters
        best_model = search.best_estimator_
        print("\nBest XGBoost parameters:")
        for param, value in best_model.named_steps['model'].get_params().items():
            print(f"{param}: {value}")
        
        # Get predictions (scaled)
        Y_pred_train = best_model.predict(split_data['X_train'])
        Y_pred_test = best_model.predict(split_data['X_test'])
        
        # Unscale predictions and true values
        Y_train_unscaled = self.ddg_scaler.inverse_transform(
            split_data['Y_train'].reshape(-1, 1)).flatten()
        Y_test_unscaled = self.ddg_scaler.inverse_transform(
            split_data['Y_test'].reshape(-1, 1)).flatten()
        Y_pred_train_unscaled = self.ddg_scaler.inverse_transform(
            Y_pred_train.reshape(-1, 1)).flatten()
        Y_pred_test_unscaled = self.ddg_scaler.inverse_transform(
            Y_pred_test.reshape(-1, 1)).flatten()
        
        # Calculate metrics
        results = {
            'train_r2': r2_score(Y_train_unscaled, Y_pred_train_unscaled),
            'test_r2': r2_score(Y_test_unscaled, Y_pred_test_unscaled),
            'train_mae': mean_absolute_error(Y_train_unscaled, Y_pred_train_unscaled),
            'test_mae': mean_absolute_error(Y_test_unscaled, Y_pred_test_unscaled),
            'model': best_model,
            'Y_pred_train': Y_pred_train_unscaled,
            'Y_pred_test': Y_pred_test_unscaled,
            'Y_train': Y_train_unscaled,
            'Y_test': Y_test_unscaled,
            'training_time': training_time,
            'best_params': best_model.get_params()
        }
        
        # Print performance metrics
        print("\nModel Performance (Original Scale):")
        print(f"Train R²: {results['train_r2']:.4f}")
        print(f"Test R²: {results['test_r2']:.4f}")
        print(f"Train MAE: {results['train_mae']:.4f} kcal/mol")
        print(f"Test MAE: {results['test_mae']:.4f} kcal/mol")
        
        # Get feature importances
        if hasattr(best_model.named_steps['model'], 'feature_importances_'):
            importances = best_model.named_steps['model'].feature_importances_
            print("\nTop 10 feature importances:")
            sorted_idx = np.argsort(importances)
            for idx in sorted_idx[-10:]:
                print(f"Feature {idx}: {importances[idx]:.4f}")
                
        # Save results and generate plots
        self.plot_results(results, split_name)
        self.save_split_info(split_data, results, split_name)
        
        return results
    
    def plot_results(self, results, split_name):
        """
        Create comprehensive visualization of model results.
        
        Args:
            results (dict): Dictionary containing model results
            split_name (str): Name identifier for the split
        """
        # Create figure with 2x2 subplots
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 20))
        
        # 1. Predictions vs Actual Scatter Plot
        ax1.scatter(
            results['Y_train'],
            results['Y_pred_train'],
            color="gray",
            alpha=0.6,
            label=f"Train (R²={results['train_r2']:.3f}, MAE={results['train_mae']:.3f})"
        )
        ax1.scatter(
            results['Y_test'],
            results['Y_pred_test'],
            color="blue",
            alpha=0.6,
            label=f"Test (R²={results['test_r2']:.3f}, MAE={results['test_mae']:.3f})"
        )
        
        # Add perfect prediction line
        min_val = min(results['Y_train'].min(), results['Y_test'].min())
        max_val = max(results['Y_train'].max(), results['Y_test'].max())
        ax1.plot([min_val, max_val], [min_val, max_val], 'k--', alpha=0.5)
        
        ax1.set_title(f"Predictions vs Actual ({split_name})")
        ax1.set_xlabel("Observed ΔΔG‡ (kcal/mol)")
        ax1.set_ylabel("Predicted ΔΔG‡ (kcal/mol)")
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # 2. Energy Distribution
        ax2.hist(results['Y_train'], bins=30, alpha=0.5, label='Train', color='gray', density=True)
        ax2.hist(results['Y_test'], bins=30, alpha=0.5, label='Test', color='blue', density=True)
        ax2.set_title("Energy Distribution")
        ax2.set_xlabel("ΔΔG‡ (kcal/mol)")
        ax2.set_ylabel("Density")
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        # 3. Residuals Plot
        train_residuals = results['Y_pred_train'] - results['Y_train']
        test_residuals = results['Y_pred_test'] - results['Y_test']
        
        ax3.scatter(results['Y_train'], train_residuals, color='gray', alpha=0.6, label='Train')
        ax3.scatter(results['Y_test'], test_residuals, color='blue', alpha=0.6, label='Test')
        ax3.axhline(y=0, color='k', linestyle='--', alpha=0.5)
        ax3.set_title("Residuals Plot")
        ax3.set_xlabel("Observed ΔΔG‡ (kcal/mol)")
        ax3.set_ylabel("Residuals (kcal/mol)")
        ax3.legend()
        ax3.grid(True, alpha=0.3)
        
        # 4. Error Distribution
        ax4.hist(train_residuals, bins=30, alpha=0.5, label='Train', color='gray', density=True)
        ax4.hist(test_residuals, bins=30, alpha=0.5, label='Test', color='blue', density=True)
        ax4.set_title("Error Distribution")
        ax4.set_xlabel("Prediction Error (kcal/mol)")
        ax4.set_ylabel("Density")
        ax4.legend()
        ax4.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(f"results_{split_name}.png", dpi=300, bbox_inches='tight')
        plt.close()
        print(f"\nPlot saved as results_{split_name}.png")

    def save_split_info(self, split_data, results, split_name):
        """
        Save detailed information about the split and results.
        
        Args:
            split_data (dict): Train/test split data
            results (dict): Model results
            split_name (str): Name identifier for the split
        """
        # Unscale ee values for reporting
        train_ee_unscaled = self.ee_scaler.inverse_transform(
            split_data['train_ee'].reshape(-1, 1)).flatten()
        test_ee_unscaled = self.ee_scaler.inverse_transform(
            split_data['test_ee'].reshape(-1, 1)).flatten()
        
        with open(f'split_info_{split_name}.txt', 'w') as f:
            f.write(f"Analysis Report: {split_name}\n")
            f.write("=" * 50 + "\n\n")
            
            # Dataset Statistics
            f.write("Dataset Statistics:\n")
            f.write("-" * 20 + "\n")
            f.write(f"Training set size: {len(split_data['X_train'])}\n")
            f.write(f"Test set size: {len(split_data['X_test'])}\n")
            f.write(f"Feature dimension: {split_data['X_train'].shape[1]}\n\n")
            
            # Energy Statistics
            f.write("Energy Statistics (kcal/mol):\n")
            f.write("-" * 20 + "\n")
            f.write(f"Training energy range: {results['Y_train'].min():.2f} to {results['Y_train'].max():.2f}\n")
            f.write(f"Test energy range: {results['Y_test'].min():.2f} to {results['Y_test'].max():.2f}\n")
            f.write(f"Training energy mean: {np.mean(results['Y_train']):.2f} ± {np.std(results['Y_train']):.2f}\n")
            f.write(f"Test energy mean: {np.mean(results['Y_test']):.2f} ± {np.std(results['Y_test']):.2f}\n\n")
            
            # Selectivity Statistics
            f.write("Selectivity Statistics (ee%):\n")
            f.write("-" * 20 + "\n")
            f.write(f"Training ee range: {train_ee_unscaled.min():.1f} to {train_ee_unscaled.max():.1f}\n")
            f.write(f"Test ee range: {test_ee_unscaled.min():.1f} to {test_ee_unscaled.max():.1f}\n")
            f.write(f"Training ee mean: {np.mean(train_ee_unscaled):.1f} ± {np.std(train_ee_unscaled):.1f}\n")
            f.write(f"Test ee mean: {np.mean(test_ee_unscaled):.1f} ± {np.std(test_ee_unscaled):.1f}\n\n")
            
            # Model Performance
            f.write("Model Performance:\n")
            f.write("-" * 20 + "\n")
            f.write(f"Training R²: {results['train_r2']:.5f}\n")
            f.write(f"Test R²: {results['test_r2']:.5f}\n")
            f.write(f"Training MAE: {results['train_mae']:.5f} kcal/mol\n")
            f.write(f"Test MAE: {results['test_mae']:.5f} kcal/mol\n")
            f.write(f"Training Time: {results['training_time']:.2f} seconds\n\n")
            
            # Best Model Parameters
            f.write("Best Model Parameters:\n")
            f.write("-" * 20 + "\n")
            for param, value in results['best_params'].items():
                f.write(f"{param}: {value}\n")
        
        print(f"\nDetailed analysis saved to split_info_{split_name}.txt")

In [3]:
def main():
    """Main function to run the complete analysis pipeline."""
    print("Starting Chemistry Analysis Pipeline...")
    print("=" * 50)
    
    # Initialize analysis
    analysis = ChemistryAwareAnalysis(
        embeddings_path='/Users/utkarsh/MMLI/equicat/develop_op/final_molecule_embeddings.json',
        data_path='/Users/utkarsh/MMLI/equicat/science/Y_DATA.csv'
    )
    
    # Prepare data
    print("\nPreparing dataset...")
    X, Y, ee_values, reaction_handles = analysis.prepare_data()
    print(f"Dataset preparation complete.")
    print(f"Total samples: {len(X)}")
    print(f"Feature dimension: {X.shape[1]}")
    
    # Perform stratified split and training
    print("\nPerforming stratified split and model training...")
    split_data = analysis.split_data_stratified(
        X, Y, ee_values, reaction_handles,
        test_fraction=0.25,  # 20% test set
        n_bins=5           # 5 energy bins
    )
    
    # Train and evaluate model
    results = analysis.train_and_evaluate(split_data, split_name="stratified_xgb")
    
    print("\nAnalysis pipeline completed!")
    print("=" * 50)
    print("\nResults Summary:")
    print(f"Training R²: {results['train_r2']:.4f}")
    print(f"Test R²: {results['test_r2']:.4f}")
    print(f"Training MAE: {results['train_mae']:.4f} kcal/mol")
    print(f"Test MAE: {results['test_mae']:.4f} kcal/mol")
    
if __name__ == "__main__":
    main()

Starting Chemistry Analysis Pipeline...
Initializing ChemistryAwareAnalysis...
Loading embeddings from /Users/utkarsh/MMLI/equicat/develop_op/final_molecule_embeddings.json
Loaded embeddings for 835 molecules
Loaded data with 1075 entries

Data Ranges (Original Scale):
ddG range: [-0.42, 3.14] kcal/mol
ee% range: [-34.00, 99.00]%

Preparing dataset...

Preparing data...
Prepared 1050 samples
Missing embeddings for 1 molecules: {'181_i'}
Embedding dimension: (768,)
Dataset preparation complete.
Total samples: 1050
Feature dimension: 768

Performing stratified split and model training...

Performing improved stratified split...

Bin statistics:

Bin 0:
Energy range: -0.42 to 0.37 kcal/mol
Number of samples: 217
Train samples: 163
Test samples: 54

Bin 1:
Energy range: 0.39 to 0.72 kcal/mol
Number of samples: 207
Train samples: 156
Test samples: 51

Bin 2:
Energy range: 0.72 to 1.18 kcal/mol
Number of samples: 217
Train samples: 163
Test samples: 54

Bin 3:
Energy range: 1.21 to 1.45 kcal